# Gunter's Space Page GSP Scraper

In [92]:
import bs4
import requests
import pandas as pd
import os

In [165]:
#url = "https://space.skyrocket.de/doc_sdat/globalstar.htm"
oneweblist = [
    "https://space.skyrocket.de/doc_sdat/oneweb.htm"
]

starlinklist = [
    "https://space.skyrocket.de/doc_sdat/starlink-v0-9.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v1-0.htm",
    "https://space.skyrocket.de/doc_sdat/starlink-v1-5.htm"
]

globalstarlist = [
    "https://space.skyrocket.de/doc_sdat/globalstar.htm",
    "https://space.skyrocket.de/doc_sdat/globalstar-2.htm"
]

iridiumlist = [
    "https://space.skyrocket.de/doc_sdat/iridium.htm",
    "https://space.skyrocket.de/doc_sdat/iridium-next.htm"
]

orbcommlist = [
    "https://space.skyrocket.de/doc_sdat/orbcomm.htm",
    "https://space.skyrocket.de/doc_sdat/orbcomm-ql.htm",
    "https://space.skyrocket.de/doc_sdat/orbcomm-2.htm"
]

swarmlist = [
    "https://space.skyrocket.de/doc_sdat/spacebee.htm",
    "https://space.skyrocket.de/doc_sdat/spacebee-5.htm",
    "https://space.skyrocket.de/doc_sdat/spacebee-10.htm"
]

urllist = oneweblist

In [166]:
def scrape_url(url):
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text)

    satdata = soup.find("table", {"id": "satdata"})
    satlist = soup.find("table", {"id": "satlist"})

    def parse_row(trrow):
        tdlist = trrow.find_all("td")
        namelist = [td.text for td in tdlist]
        return namelist

    satlist_dict = {
        "Satellite": [],
        "COSPAR": [],
        "Date": [],
        "LS": [],
        "Failed": [],
        "Launche Vehicle": [],
        "Remarks": []
    }

    for trrow in satlist.find_all("tr")[1:]:
        #if
        namelist = parse_row(trrow)
        datestr = namelist[2].lower()
        if(datestr == "cancelled" or datestr == "2023" or datestr == "not launched"):
            continue
        try:
            date = pd.to_datetime(datestr, format="%d.%m.%Y")
        except:
            continue
        satlist_dict["Satellite"].append(namelist[0])
        satlist_dict["COSPAR"].append(namelist[1])
        satlist_dict["Date"].append(date)
        satlist_dict["LS"].append(namelist[3])
        satlist_dict["Failed"].append(namelist[4])
        satlist_dict["Launche Vehicle"].append(namelist[5])
        satlist_dict["Remarks"].append(namelist[6])

    df = pd.DataFrame(satlist_dict)
    return df

dflist = [scrape_url(url) for url in urllist]
df = pd.concat(dflist)

In [167]:
print("Count", len(df.groupby(by="Date").count()))
print(df.groupby(by="Date").count().index)

Count 16
DatetimeIndex(['2019-02-27', '2020-02-06', '2020-03-21', '2020-12-18',
               '2021-03-25', '2021-04-25', '2021-05-28', '2021-07-01',
               '2021-08-21', '2021-09-14', '2021-10-14', '2021-12-27',
               '2022-02-10', '2022-10-22', '2022-12-08', '2023-01-10'],
              dtype='datetime64[ns]', name='Date', freq=None)


In [168]:
len(df)

544

In [58]:
#df.to_csv("parsed_gsp/{}.csv".format("globalstar"))

In [54]:
#df.to_csv("parsed_gsp/{}.csv".format(url.split("/")[-1]))